In [3]:
%pip install pyspark==3.3.1


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
spark = (SparkSession.builder
         .appName('GOLD_TRANSFORM_T')
         .getOrCreate()
         )

24/09/03 21:29:54 WARN Utils: Your hostname, kaiwangairflow resolves to a loopback address: 127.0.1.1; using 192.168.2.128 instead (on interface ens33)
24/09/03 21:29:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/09/03 21:29:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
twitter_df = spark.read.json("../../datalake/silver/data_engineer/twitter_df")
twitter_df.show()

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|process_data|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+------------+
|       60|             79|2024-08-27T13:28:...| 58|        12|         88|          4|           16|Este é um tweet f...|    20240827|
|       48|             37|2024-08-27T13:45:...| 65|        38|         76|         53|           16|Outro tweet fictí...|    20240827|
|        9|             34|2024-08-27T23:37:...| 66|        27|         67|         96|           45|Este é um tweet f...|    20240827|
|       63|             25|2024-08-27T20:26:...| 40|        99|         52|         82|            9|Tweet fictício cr...|    20240827|
|       29|             51|2024-08-27T10:11:...|

In [4]:
rank_tweet = (twitter_df
              .alias('tweet')
              .groupBy(f.to_date('created_at').alias("create_date")).agg(
                    f.countDistinct('author_id').alias('n_usuarios'),
                    f.sum('like_count').alias('n_like'),
                    f.sum('quote_count').alias('n_quote'),
                    f.sum('reply_count').alias('n_reply'),
                    f.sum('retweet_count').alias('n_retweet')
                ).withColumn('weekday', f.date_format('create_date', 'E')))

rank_tweet.show()

+-----------+----------+------+-------+-------+---------+-------+
|create_date|n_usuarios|n_like|n_quote|n_reply|n_retweet|weekday|
+-----------+----------+------+-------+-------+---------+-------+
| 2024-08-27|       101| 50683|  50721|  49669|    50369|    Tue|
| 2024-08-30|       101| 48237|  49790|  48952|    49033|    Fri|
| 2024-08-29|       101| 51141|  50796|  49886|    51928|    Thu|
| 2024-09-03|       101| 50107|  48395|  49080|    50704|    Tue|
| 2024-09-01|        10|   508|    525|    553|      548|    Sun|
| 2024-08-25|       101| 49700|  50407|  50549|    51976|    Sun|
| 2024-09-02|         9|   500|    411|    380|      445|    Mon|
| 2024-08-19|        10|   622|    650|    516|      511|    Mon|
| 2024-08-24|       101| 48924|  50231|  49560|    52036|    Sat|
| 2024-08-23|       101| 48580|  50821|  48759|    50210|    Fri|
| 2024-08-31|         9|   471|    489|    480|      524|    Sat|
| 2024-08-21|        10|   545|    551|    388|      397|    Wed|
| 2024-08-

In [5]:
rank_tweet.coalesce(1).write.mode('overwrite').json('../../datalake/gold/data_engineer/twitter_df')